In [162]:
import pandas as pd
import sqlite3 as lite

def create_connection(db_file):
    conn = None
    try:
        conn = lite.connect(db_file, timeout=10)
    except Error as e:
        print(e)
    return conn

conn = create_connection("assets/CVEfixes.db")

In [163]:
import pandas as pd

query = """
    SELECT *
    FROM file_change
"""

df = pd.read_sql_query(query, conn)

df.head()

,file_change_id,hash,filename,old_path,new_path,change_type,diff,diff_parsed,num_lines_added,num_lines_deleted,code_after,code_before,nloc,complexity,token_count,programming_language
0,100562011220946,2864e767053317538feafa815046fff89e5a16be,ChangeLog,ChangeLog,ChangeLog,ModificationType.MODIFY,"@@ -1,3 +1,55 @@\n+1999-11-09 Ulrich Drepper ...","{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,nan,nan,nan,Batchfile
1,7461293048740,2864e767053317538feafa815046fff89e5a16be,dl-load.c,elf/dl-load.c,elf/dl-load.c,ModificationType.MODIFY,"@@ -149,21 +149,31 @@ local_strdup (const char...","{'added': [(152, ' const char *const start = ...",28,10,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,952.0,260.0,6592.0,C
2,169254254633061,2864e767053317538feafa815046fff89e5a16be,search.texi,manual/search.texi,manual/search.texi,ModificationType.MODIFY,"@@ -475,9 +475,11 @@ elements.\n To remove a s...","{'added': [(478, 'and a pointer to the parent ...",5,3,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...",nan,nan,nan,TeX
3,253444729592566,2864e767053317538feafa815046fff89e5a16be,syslog.c,misc/syslog.c,misc/syslog.c,ModificationType.MODIFY,"@@ -177,10 +177,14 @@ vsyslog(pri, fmt, ap)\n ...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...",229.0,42.0,1240.0,C
4,197890948684264,2864e767053317538feafa815046fff89e5a16be,Makefile,posix/Makefile,posix/Makefile,ModificationType.MODIFY,"@@ -57,7 +57,7 @@ include ../Makeconfig\n \n a...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...",nan,nan,nan,Shell


In [164]:
df.programming_language.value_counts()

programming_language
PHP                 5841
C                   5214
unknown             3792
JavaScript          2004
C++                 1720
Markdown            1495
Python              1208
Java                1187
Ruby                1021
HTML                 870
TypeScript           811
Go                   668
Shell                667
SQL                  347
C#                   283
Batchfile            276
CoffeeScript         268
Objective-C          254
Perl                 228
CSS                  220
Scala                194
Lua                  181
Rust                 118
TeX                   91
Haskell               89
PowerShell            68
Jupyter Notebook      63
Swift                 54
Matlab                43
R                     20
Erlang                 9
None                   5
Name: count, dtype: int64

In [165]:
df = df[['filename', 'programming_language', 'code_before', 'code_after',
         'diff_parsed', 'num_lines_added', 'num_lines_deleted', 'nloc', 'token_count', 'complexity',
         'file_change_id', 'hash', 'change_type', 'old_path', 'new_path']]

df = df.rename(columns={
    'filename': 'file_name',
    'old_path': 'old_file_path',
    'new_path': 'new_file_path',
    'diff_parsed': 'diff',
    'nloc': 'num_lines_in_file',
    'token_count': 'num_tokens_in_file',  
})

df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,file_change_id,hash,change_type,old_file_path,new_file_path
0,ChangeLog,Batchfile,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,"{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,nan,nan,nan,100562011220946,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,ChangeLog,ChangeLog
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,7461293048740,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,elf/dl-load.c,elf/dl-load.c
2,search.texi,TeX,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...","{'added': [(478, 'and a pointer to the parent ...",5,3,nan,nan,nan,169254254633061,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,manual/search.texi,manual/search.texi
3,syslog.c,C,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,229.0,1240.0,42.0,253444729592566,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,misc/syslog.c,misc/syslog.c
4,Makefile,Shell,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,nan,nan,nan,197890948684264,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,posix/Makefile,posix/Makefile


In [166]:
query = """
    SELECT *
    FROM commits
"""

# Using pandas to read data from the database and store it into a DataFrame
df1 = pd.read_sql_query(query, conn)

df1 = df1[[
    'hash', 'repo_url', 'author', 'committer', 'msg', 'merge'
]]
df1 = df1.rename(columns={
    'msg': 'commit_message',
})

merged_df = df.merge(df1, on='hash', how='inner')

# Display the merged DataFrame
merged_df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,file_change_id,hash,change_type,old_file_path,new_file_path,repo_url,author,committer,commit_message,merge
0,ChangeLog,Batchfile,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,"{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,nan,nan,nan,100562011220946,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,ChangeLog,ChangeLog,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,7461293048740,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,elf/dl-load.c,elf/dl-load.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False
2,search.texi,TeX,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...","{'added': [(478, 'and a pointer to the parent ...",5,3,nan,nan,nan,169254254633061,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,manual/search.texi,manual/search.texi,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False
3,syslog.c,C,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,229.0,1240.0,42.0,253444729592566,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,misc/syslog.c,misc/syslog.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False
4,Makefile,Shell,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,nan,nan,nan,197890948684264,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,posix/Makefile,posix/Makefile,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False


In [167]:
query = """
    SELECT *
    FROM fixes
"""

# Using pandas to read data from the database and store it into a DataFrame
df3 = pd.read_sql_query(query, conn)

df3 = df3[[
    'cve_id', 'hash'
]]

# df2 = df2.rename(columns={
#     'stars_count': 'repo_stars_count',
# })

merged_df = merged_df.merge(df3, on='hash', how='inner')

# Display the merged DataFrame
merged_df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,hash,change_type,old_file_path,new_file_path,repo_url,author,committer,commit_message,merge,cve_id
0,ChangeLog,Batchfile,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,"{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,nan,nan,nan,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,ChangeLog,ChangeLog,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,elf/dl-load.c,elf/dl-load.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199
2,search.texi,TeX,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...","{'added': [(478, 'and a pointer to the parent ...",5,3,nan,nan,nan,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,manual/search.texi,manual/search.texi,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199
3,syslog.c,C,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,229.0,1240.0,42.0,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,misc/syslog.c,misc/syslog.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199
4,Makefile,Shell,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,nan,nan,nan,...,2864e767053317538feafa815046fff89e5a16be,ModificationType.MODIFY,posix/Makefile,posix/Makefile,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199


In [168]:
query = """
    SELECT *
    FROM cwe_classification
"""

# Using pandas to read data from the database and store it into a DataFrame
df4 = pd.read_sql_query(query, conn)

df4 = df4.groupby('cve_id')['cwe_id'].apply(list).reset_index()

df4.head(100)

,cve_id,cwe_id
0,CVE-1999-0199,[CWE-252]
1,CVE-1999-0731,[NVD-CWE-Other]
2,CVE-2002-2443,[CWE-20]
3,CVE-2006-3635,[CWE-119]
4,CVE-2006-5331,[CWE-19]
...,...,...
95,CVE-2011-2905,[NVD-CWE-Other]
96,CVE-2011-2906,[CWE-400]
97,CVE-2011-2909,[CWE-200]
98,CVE-2011-2918,[CWE-400]


In [169]:
merged_df = merged_df.merge(df4, on='cve_id', how='inner')
merged_df = merged_df.explode('cwe_id').reset_index(drop=True)

# Display the merged DataFrame
merged_df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,change_type,old_file_path,new_file_path,repo_url,author,committer,commit_message,merge,cve_id,cwe_id
0,ChangeLog,Batchfile,1999-11-08 Ulrich Drepper <drepper@cygnus.co...,1999-11-09 Ulrich Drepper <drepper@cygnus.co...,"{'added': [(1, '1999-11-09 Ulrich Drepper <d...",52,0,nan,nan,nan,...,ModificationType.MODIFY,ChangeLog,ChangeLog,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199,CWE-252
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,ModificationType.MODIFY,elf/dl-load.c,elf/dl-load.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199,CWE-252
2,search.texi,TeX,"@node Searching and Sorting, Pattern Matching,...","@node Searching and Sorting, Pattern Matching,...","{'added': [(478, 'and a pointer to the parent ...",5,3,nan,nan,nan,...,ModificationType.MODIFY,manual/search.texi,manual/search.texi,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199,CWE-252
3,syslog.c,C,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,229.0,1240.0,42.0,...,ModificationType.MODIFY,misc/syslog.c,misc/syslog.c,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199,CWE-252
4,Makefile,Shell,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","{'added': [(60, '\t\t tst-preadwrite test-vf...",1,1,nan,nan,nan,...,ModificationType.MODIFY,posix/Makefile,posix/Makefile,https://github.com/bminor/glibc,Ulrich Drepper,Ulrich Drepper,Update.\n\n1999-11-09 Ulrich Drepper <dreppe...,False,CVE-1999-0199,CWE-252


In [170]:
query = """
    SELECT *
    FROM method_change
"""

# Using pandas to read data from the database and store it into a DataFrame
df6 = pd.read_sql_query(query, conn)

df6 = df6[[
    'file_change_id', 'method_change_id', 'name', 'signature', 'parameters', 'code', 'nloc', 'complexity', 'token_count', 'before_change'
]]

df6 = df6.rename(columns={
    'name': 'method_name',
    'signature': 'method_signature', 
    'parameters': 'method_parameters',
    'code': 'method_code',
    'nloc': 'num_lines_in_method',
    'complexity': 'method_complexity',
    'token_count': 'num_tokens_in_method',
    'before_change': 'vulnerable'
})

In [171]:
merged_df = merged_df.merge(df6, on='file_change_id', how='inner')

# Display the merged DataFrame
merged_df.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,cwe_id,method_change_id,method_name,method_signature,method_parameters,method_code,num_lines_in_method,method_complexity,num_tokens_in_method,vulnerable
0,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,241941308837203,expand_dynamic_string_token,expand_dynamic_string_token( struct link_map *...,"['l', 's']",expand_dynamic_string_token (struct link_map *...,14,3,95,True
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,235368100398254,_dl_dst_substitute,"_dl_dst_substitute( struct link_map * l , cons...","['l', 'name', 'result', 'is_path']","_dl_dst_substitute (struct link_map *l, const ...",51,22,349,True
2,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,161060042440601,_dl_dst_count,"_dl_dst_count( const char * name , int is_path)","['name', 'is_path']","_dl_dst_count (const char *name, int is_path)\...",22,13,199,True
3,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,205987001054463,expand_dynamic_string_token,expand_dynamic_string_token( struct link_map *...,"['l', 's']",expand_dynamic_string_token (struct link_map *...,14,3,95,False
4,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,38197852149195,_dl_dst_substitute,"_dl_dst_substitute( struct link_map * l , cons...","['l', 'name', 'result', 'is_path']","_dl_dst_substitute (struct link_map *l, const ...",57,28,399,False


In [172]:
merged_df.columns

Index(['file_name', 'programming_language', 'code_before', 'code_after',
       'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file',
       'num_tokens_in_file', 'complexity', 'file_change_id', 'hash',
       'change_type', 'old_file_path', 'new_file_path', 'repo_url', 'author',
       'committer', 'commit_message', 'merge', 'cve_id', 'cwe_id',
       'method_change_id', 'method_name', 'method_signature',
       'method_parameters', 'method_code', 'num_lines_in_method',
       'method_complexity', 'num_tokens_in_method', 'vulnerable'],
      dtype='object')

In [173]:
merged_df.programming_language.value_counts()

programming_language
JavaScript      46755
PHP             18187
C               14474
unknown         13635
C++              5101
Python           4343
Java             3955
Ruby             3064
Go               2274
TypeScript       1193
C#                942
Objective-C       643
HTML              298
Scala             294
Rust              264
Swift             118
Lua                97
CoffeeScript       80
Shell              69
SQL                67
Perl               67
PowerShell         47
Matlab             39
Markdown           32
TeX                10
Haskell             6
Erlang              4
Batchfile           2
None                1
Name: count, dtype: int64

In [174]:
merged_df.columns

Index(['file_name', 'programming_language', 'code_before', 'code_after',
       'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file',
       'num_tokens_in_file', 'complexity', 'file_change_id', 'hash',
       'change_type', 'old_file_path', 'new_file_path', 'repo_url', 'author',
       'committer', 'commit_message', 'merge', 'cve_id', 'cwe_id',
       'method_change_id', 'method_name', 'method_signature',
       'method_parameters', 'method_code', 'num_lines_in_method',
       'method_complexity', 'num_tokens_in_method', 'vulnerable'],
      dtype='object')

In [176]:
me_c_cpp = merged_df[(merged_df['programming_language'] == 'C') | (merged_df['programming_language'] == 'C++')]
me_c_cpp_shape = me_c_cpp.shape
me_c_cpp_shape

(19575, 31)

In [178]:
me_c_cpp.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,cwe_id,method_change_id,method_name,method_signature,method_parameters,method_code,num_lines_in_method,method_complexity,num_tokens_in_method,vulnerable
0,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,241941308837203,expand_dynamic_string_token,expand_dynamic_string_token( struct link_map *...,"['l', 's']",expand_dynamic_string_token (struct link_map *...,14,3,95,True
1,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,235368100398254,_dl_dst_substitute,"_dl_dst_substitute( struct link_map * l , cons...","['l', 'name', 'result', 'is_path']","_dl_dst_substitute (struct link_map *l, const ...",51,22,349,True
2,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,161060042440601,_dl_dst_count,"_dl_dst_count( const char * name , int is_path)","['name', 'is_path']","_dl_dst_count (const char *name, int is_path)\...",22,13,199,True
3,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,205987001054463,expand_dynamic_string_token,expand_dynamic_string_token( struct link_map *...,"['l', 's']",expand_dynamic_string_token (struct link_map *...,14,3,95,False
4,dl-load.c,C,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,"{'added': [(152, ' const char *const start = ...",28,10,952.0,6592.0,260.0,...,CWE-252,38197852149195,_dl_dst_substitute,"_dl_dst_substitute( struct link_map * l , cons...","['l', 'name', 'result', 'is_path']","_dl_dst_substitute (struct link_map *l, const ...",57,28,399,False


In [180]:
me_c_cpp.columns

Index(['file_name', 'programming_language', 'code_before', 'code_after',
       'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file',
       'num_tokens_in_file', 'complexity', 'file_change_id', 'hash',
       'change_type', 'old_file_path', 'new_file_path', 'repo_url', 'author',
       'committer', 'commit_message', 'merge', 'cve_id', 'cwe_id',
       'method_change_id', 'method_name', 'method_signature',
       'method_parameters', 'method_code', 'num_lines_in_method',
       'method_complexity', 'num_tokens_in_method', 'vulnerable'],
      dtype='object')

In [181]:
me_c_cpp.vulnerable.value_counts()

vulnerable
False    11349
True      8226
Name: count, dtype: int64

In [ ]:
me_c_cpp.to_csv('CVEFixes_c.csv', index=False)

In [12]:
me_py = merged_df[merged_df.programming_language == 'Python']
me_py.shape

(4201, 31)

In [13]:
me_py.to_csv('CVEFixes_python.csv', index=False)

In [14]:
me_c = merged_df[merged_df.programming_language == 'C']
me_c.shape

(13840, 31)

In [15]:
me_c.to_csv('CVEFixes_c.csv', index=False)

In [16]:
me_ja = merged_df[merged_df.programming_language == 'Java']
me_ja.shape

(3855, 31)

In [17]:
me_ja.to_csv('CVEFixes_java.csv', index=False)

In [19]:
me_cpp = merged_df[merged_df.programming_language == 'C++']
me_cpp.shape

(4393, 31)

In [20]:
me_cpp.to_csv('CVEFixes_c++.csv', index=False)

In [247]:
me_ru = merged_df[merged_df.programming_language == 'Ruby']
me_ru.shape

(3056, 31)

In [248]:
me_ru.to_csv('CVEFixes_ruby.csv', index=False)

In [249]:
me_go = merged_df[merged_df.programming_language == 'Go']
me_go.shape

(2274, 31)

In [250]:
me_go.to_csv('CVEFixes_go.csv', index=False)

In [80]:
merged_df[merged_df['file_change_id']=='7461293048740'].file_change_id

0    7461293048740
1    7461293048740
2    7461293048740
3    7461293048740
4    7461293048740
5    7461293048740
Name: file_change_id, dtype: object

In [36]:
print(merged_df['file_change_id'].iloc[1])

7461293048740


In [182]:
merged_df = me_c_cpp

In [183]:
me1 = merged_df.drop(['committer', 'author', 'file_change_id', 'hash'], axis=1)
me1.columns

Index(['file_name', 'programming_language', 'code_before', 'code_after',
       'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file',
       'num_tokens_in_file', 'complexity', 'change_type', 'old_file_path',
       'new_file_path', 'repo_url', 'commit_message', 'merge', 'cve_id',
       'cwe_id', 'method_change_id', 'method_name', 'method_signature',
       'method_parameters', 'method_code', 'num_lines_in_method',
       'method_complexity', 'num_tokens_in_method', 'vulnerable'],
      dtype='object')

In [294]:
me_py = pd.read_csv('CVEFixes_c++.csv')

In [295]:
me_py.head()

,file_name,programming_language,code_before,code_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,complexity,...,cwe_id,method_change_id,method_name,method_signature,method_parameters,method_code,num_lines_in_method,method_complexity,num_tokens_in_method,vulnerable
0,saver.cpp,C++,#include <pwd.h>\n#include <unistd.h>\n#includ...,#include <pwd.h>\n#include <unistd.h>\n#includ...,"{'added': [(154, ' timer.stop();')]...",1.0,0.0,197.0,1132.0,34.0,...,['NVD-CWE-Other'],2.759389e+13,KPasswordDlg::keyPressed,KPasswordDlg::keyPressed( QKeyEvent * e),['e'],void KPasswordDlg::keyPressed( QKeyEvent *e )\...,44.0,10.0,184.0,False
1,perf_event_v6.c,C++,/*\n * ARMv6 Performance counter handling code...,/*\n * ARMv6 Performance counter handling code...,"{'added': [(482, '\t\tif (perf_event_overflow(...",1.0,1.0,498.0,2757.0,45.0,...,['CWE-400'],1.609609e+14,armv6pmu_handle_irq,"armv6pmu_handle_irq( int irq_num , void * dev)","['irq_num', 'dev']","armv6pmu_handle_irq(int irq_num,\n\t\t void...",32.0,7.0,206.0,True
2,perf_event_v6.c,C++,/*\n * ARMv6 Performance counter handling code...,/*\n * ARMv6 Performance counter handling code...,"{'added': [(482, '\t\tif (perf_event_overflow(...",1.0,1.0,498.0,2757.0,45.0,...,['CWE-400'],9.293469e+13,armv6pmu_handle_irq,"armv6pmu_handle_irq( int irq_num , void * dev)","['irq_num', 'dev']","armv6pmu_handle_irq(int irq_num,\n\t\t void...",32.0,7.0,204.0,False
3,perf_event_intel.c,C++,#ifdef CONFIG_CPU_SUP_INTEL\n\n#define MAX_EXT...,#ifdef CONFIG_CPU_SUP_INTEL\n\n#define MAX_EXT...,"{'added': [(1006, '\t\tif (perf_event_overflow...",1.0,1.0,1132.0,6756.0,121.0,...,['CWE-400'],4.727234e+13,intel_pmu_handle_irq,intel_pmu_handle_irq( struct pt_regs * regs),['regs'],static int intel_pmu_handle_irq(struct pt_regs...,50.0,8.0,266.0,True
4,perf_event_intel.c,C++,#ifdef CONFIG_CPU_SUP_INTEL\n\n#define MAX_EXT...,#ifdef CONFIG_CPU_SUP_INTEL\n\n#define MAX_EXT...,"{'added': [(1006, '\t\tif (perf_event_overflow...",1.0,1.0,1132.0,6756.0,121.0,...,['CWE-400'],1.460641e+14,intel_pmu_handle_irq,intel_pmu_handle_irq( struct pt_regs * regs),['regs'],static int intel_pmu_handle_irq(struct pt_regs...,50.0,8.0,264.0,False


In [296]:
me_py.columns

Index(['file_name', 'programming_language', 'code_before', 'code_after',
       'diff', 'num_lines_added', 'num_lines_deleted', 'num_lines_in_file',
       'num_tokens_in_file', 'complexity', 'file_change_id', 'hash',
       'change_type', 'old_file_path', 'new_file_path', 'repo_url', 'author',
       'committer', 'commit_message', 'merge', 'cve_id', 'cwe_id',
       'method_change_id', 'method_name', 'method_signature',
       'method_parameters', 'method_code', 'num_lines_in_method',
       'method_complexity', 'num_tokens_in_method', 'vulnerable'],
      dtype='object')

In [297]:
me_py.vulnerable.value_counts()

vulnerable
False    2643
True     1746
Name: count, dtype: int64

In [15]:
merged_df.shape

(116061, 31)

In [184]:
filtered_df = merged_df.groupby('file_change_id').filter(lambda x: len(x) >= 2)
filtered_df = filtered_df.groupby(['file_change_id', 'method_name', 'hash']).filter(lambda x: len(x) >= 2)

def process_group(group):
    if len(group) >= 2 and group['vulnerable'].nunique() == 2:
        function_before = group.loc[group['vulnerable'] == "True"].iloc[0]
        function_after = group.loc[group['vulnerable'] == "False"].iloc[0]

        return pd.Series({
            'file_name': function_before['file_name'],
            'programming_language': function_before['programming_language'],
            'method_name': function_before['method_name'],
            'code_before': function_before['code_before'],
            'code_after': function_before['code_after'],
            'func_before': function_before['method_code'],
            'func_after': function_after['method_code'],
            'diff': function_before['diff'],
            'num_lines_added': function_before['num_lines_added'],
            'num_lines_deleted': function_before['num_lines_deleted'],
            'num_lines_in_file': function_before['num_lines_in_file'],
            'num_tokens_in_file': function_before['num_tokens_in_file'],
            'num_lines_in_method': function_before['num_lines_in_method'],
            'num_tokens_in_method': function_before['num_tokens_in_method'],
            'method_complexity': function_before['method_complexity'],
            'repo': function_before['repo_url'],
            'cve_id': function_before['cve_id'],
            'cwe_id': function_before['cwe_id'],
        })

# Apply the function to each group and drop any None results
result_df = filtered_df.groupby(['file_change_id', 'method_name', 'hash']).apply(process_group).dropna().reset_index(drop=True)

/tmp/ipykernel_174155/1159927496.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = filtered_df.groupby(['file_change_id', 'method_name', 'hash']).apply(process_group).dropna().reset_index(drop=True)


In [185]:
result_df.shape

(5937, 18)

In [186]:
result_df.head()

,file_name,programming_language,method_name,code_before,code_after,func_before,func_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,num_lines_in_method,num_tokens_in_method,method_complexity,repo,cve_id,cwe_id
0,check_securechannel.c,C,setup_secureChannel,/* This Source Code Form is subject to the ter...,/* This Source Code Form is subject to the ter...,setup_secureChannel(void) {\n TestingPolicy...,setup_secureChannel(void) {\n TestingPolicy...,"{'added': [(48, ' testingConnection ='), (4...",2,1,436,3007,9,73,1,https://github.com/open62541/open62541,CVE-2022-25761,CWE-770
1,codegen.c,C,gen_assignment,/*\n** codegen.c - mruby code generator\n**\n*...,/*\n** codegen.c - mruby code generator\n**\n*...,"gen_assignment(codegen_scope *s, node *tree, n...","gen_assignment(codegen_scope *s, node *tree, n...","{'added': [(1907, ' if (n < 15) {'), (190...",5,1,3530,23087,162,918,48,https://github.com/mruby/mruby,CVE-2022-0717,CWE-125
2,valid.c,C,xmlAddID,/*\n * valid.c : part of the code use to do th...,/*\n * valid.c : part of the code use to do th...,"xmlAddID(xmlValidCtxtPtr ctxt, xmlDocPtr doc, ...","xmlAddID(xmlValidCtxtPtr ctxt, xmlDocPtr doc, ...","{'added': [(482, '/**'), (483, ' * xmlValidNor...",55,33,4809.0,30269.0,52,320,14,https://github.com/GNOME/libxml2,CVE-2022-23308,CWE-416
3,valid.c,C,xmlAddRef,/*\n * valid.c : part of the code use to do th...,/*\n * valid.c : part of the code use to do th...,"xmlAddRef(xmlValidCtxtPtr ctxt, xmlDocPtr doc,...","xmlAddRef(xmlValidCtxtPtr ctxt, xmlDocPtr doc,...","{'added': [(482, '/**'), (483, ' * xmlValidNor...",55,33,4809.0,30269.0,69,393,16,https://github.com/GNOME/libxml2,CVE-2022-23308,CWE-416
4,valid.c,C,xmlValidCtxtNormalizeAttributeValue,/*\n * valid.c : part of the code use to do th...,/*\n * valid.c : part of the code use to do th...,xmlValidCtxtNormalizeAttributeValue(xmlValidCt...,xmlValidCtxtNormalizeAttributeValue(xmlValidCt...,"{'added': [(482, '/**'), (483, ' * xmlValidNor...",55,33,4809.0,30269.0,60,473,29,https://github.com/GNOME/libxml2,CVE-2022-23308,CWE-416


In [187]:
result_df.cwe_id.value_counts()

cwe_id
CWE-125    918
CWE-119    545
CWE-787    468
CWE-20     383
CWE-476    324
          ... 
CWE-428      1
CWE-943      1
CWE-116      1
CWE-320      1
CWE-94       1
Name: count, Length: 115, dtype: int64

In [190]:
att_id = [787, 79, 89, 416, 78, 20, 125, 22, 352, 434, 862, 476, 287, 190, 502, 77, 119, 798, 918, 306, 362, 269, 94, 863, 276]
attention_cwe = [f'CWE-{i}' for i in att_id]
tmp_df = result_df[result_df['cwe_id'].isin(attention_cwe)]
tmp_df.shape

(3512, 18)

In [197]:
cwe_counts = tmp_df['cwe_id'].value_counts()

# Filter 'cwe_id' to keep only those with less than 150 occurrences
cwe_to_keep = cwe_counts[cwe_counts > 150].index

# Filter the DataFrame based on 'cwe_id' counts
filtered_tmp_df = tmp_df[tmp_df['cwe_id'].isin(cwe_to_keep)]

# Adding 'id' column as the first column again in case it needs to be re-added
filtered_tmp_df['id'] = range(filtered_tmp_df.shape[0])
cols = ['id'] + [col for col in filtered_tmp_df.columns if col != 'id']
filtered_tmp_df = filtered_tmp_df[cols]

/tmp/ipykernel_174155/389181450.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tmp_df['id'] = range(filtered_tmp_df.shape[0])


In [198]:
filtered_tmp_df.shape

(3289, 19)

In [199]:
filtered_tmp_df.cwe_id.value_counts()

cwe_id
CWE-125    918
CWE-119    545
CWE-787    468
CWE-20     383
CWE-476    324
CWE-190    250
CWE-416    207
CWE-362    194
Name: count, dtype: int64

In [200]:
filtered_tmp_df.columns

Index(['id', 'file_name', 'programming_language', 'method_name', 'code_before',
       'code_after', 'func_before', 'func_after', 'diff', 'num_lines_added',
       'num_lines_deleted', 'num_lines_in_file', 'num_tokens_in_file',
       'num_lines_in_method', 'num_tokens_in_method', 'method_complexity',
       'repo', 'cve_id', 'cwe_id'],
      dtype='object')

In [201]:
filtered_tmp_df.to_csv("cveFixes_master_c.csv", index=False)

In [ ]:
tmp_df['id'] = range(tmp_df.shape[0])

# Reorder the columns to place 'id' first
cols = ['id'] + [col for col in tmp_df.columns if col != 'id']
tmp_df = tmp_df[cols]

In [112]:
cwe20 = result_df[result_df['cwe_id']=='CWE-20']
cwe20.shape

(1855, 18)

In [113]:
cwe20.programming_language.value_counts()

programming_language
JavaScript     1120
C               287
PHP             157
C++              96
Python           66
Ruby             54
Java             24
TypeScript       24
Go               10
Objective-C       7
C#                6
Rust              1
SQL               1
Haskell           1
Lua               1
Name: count, dtype: int64

In [114]:
cwe20.columns

Index(['file_name', 'programming_language', 'method_name', 'code_before',
       'code_after', 'func_before', 'func_after', 'diff', 'num_lines_added',
       'num_lines_deleted', 'num_lines_in_file', 'num_tokens_in_file',
       'num_lines_in_method', 'num_tokens_in_method', 'method_complexity',
       'repo', 'cve_id', 'cwe_id'],
      dtype='object')

In [115]:
import ast
# cwe20.num_tokens_in_file = cwe20.num_tokens_in_file.apply(ast.literal_eval)
# cwe20.num_lines_in_file = cwe20.num_lines_in_file.apply(str)
# cwe20.num_lines_in_file = cwe20.num_lines_in_file.apply(ast.literal_eval)
cwe20.num_lines_in_method = cwe20.num_lines_in_method.apply(ast.literal_eval)

/tmp/ipykernel_174155/3530366171.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cwe20.num_lines_in_method = cwe20.num_lines_in_method.apply(ast.literal_eval)


In [66]:
cwe20.num_tokens_in_file.describe()

count    1.855000e+03
mean     9.398365e+05
std      9.331303e+05
min      5.700000e+01
25%      3.157000e+03
50%      6.038870e+05
75%      1.471078e+06
max      2.382875e+06
Name: num_tokens_in_file, dtype: float64

In [67]:
cwe20.num_lines_in_file.describe()

count     1855.000000
mean      7708.246900
std       7086.544517
min          1.000000
25%        511.000000
50%       7043.000000
75%      13252.000000
max      17975.000000
Name: num_lines_in_file, dtype: float64

In [157]:
cwe21 = cwe20[(cwe20['num_lines_in_method'] < 40)]
cwe21.shape

(1597, 18)

In [158]:
cwe21.programming_language.value_counts()

programming_language
JavaScript     1113
PHP             144
C               135
Python           54
Ruby             53
C++              52
Java             20
TypeScript       17
Go                6
Objective-C       2
C#                1
Name: count, dtype: int64

In [144]:
cwe_sub = cwe21[(cwe21['programming_language'].str.contains('|'.join(['Ruby', 'PHP', 'Python', 'C'])))]
cwe_sub.shape

(493, 18)

In [145]:
cwe_sub.programming_language.value_counts()

programming_language
C              167
PHP            151
C++             59
Python          57
Ruby            54
C#               3
Objective-C      2
Name: count, dtype: int64

In [150]:
cwe_sub = cwe_sub[cwe_sub['programming_language'] != 'Objective-C']
cwe_sub = cwe_sub[cwe_sub['programming_language'] != 'JavaScript']
cwe_sub = cwe_sub[cwe_sub['programming_language'] != 'C#']

In [154]:
min_lines_per_language = cwe_sub.groupby('programming_language')['num_lines_in_method'].min()

cwe_smallest = pd.concat([cwe_sub[cwe_sub['programming_language'] == lang].nsmallest(50, 'num_lines_in_method') for lang in min_lines_per_language.index])

In [159]:
cwe_smallest.programming_language.value_counts()

programming_language
C         50
C++       50
PHP       50
Python    50
Ruby      50
Name: count, dtype: int64

In [161]:
cwe_smallest.to_csv('q3_test.csv', index=False)

In [132]:
cwe_smallest.head()

,file_name,programming_language,method_name,code_before,code_after,func_before,func_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,num_lines_in_method,num_tokens_in_method,method_complexity,repo,cve_id,cwe_id
16189,nf_conntrack_proto_generic.c,C,generic_new,/* (C) 1999-2001 Paul `Rusty' Russell\n * (C) ...,/* (C) 1999-2001 Paul `Rusty' Russell\n * (C) ...,"static bool generic_new(struct nf_conn *ct, co...","static bool generic_new(struct nf_conn *ct, co...","{'added': [(17, 'static bool nf_generic_should...",25,1,173.0,865.0,5,27,1,https://github.com/torvalds/linux,CVE-2014-8160,CWE-20
534,jas_icc.c,C,jas_iccputsint,/*\n * Copyright (c) 2002-2003 Michael David A...,/*\n * Copyright (c) 2002-2003 Michael David A...,"static int jas_iccputsint(jas_stream_t *out, i...","static int jas_iccputsint(jas_stream_t *out, i...","{'added': [(85, 'static int jas_iccgetuint(jas...",15,15,1502.0,10066.0,6,45,2,https://github.com/mdadams/jasper,CVE-2016-9395,CWE-20
5502,segment.c,C,f2fs_wait_discard_bios,/*\n * fs/f2fs/segment.c\n *\n * Copyright (c)...,/*\n * fs/f2fs/segment.c\n *\n * Copyright (c)...,void f2fs_wait_discard_bios(struct f2fs_sb_inf...,void f2fs_wait_discard_bios(struct f2fs_sb_inf...,"{'added': [(1213, 'void f2fs_wait_discard_bios...",3,3,2836.0,18721.0,6,28,1,https://github.com/torvalds/linux,CVE-2017-18200,CWE-20
25464,user_defined.c,C,user_describe,/* user_defined.c: user defined key type\n *\n...,/* user_defined.c: user defined key type\n *\n...,"void user_describe(const struct key *key, stru...","void user_describe(const struct key *key, stru...","{'added': [(117, '\tif (key_is_positive(key))'...",2,2,125.0,699.0,6,42,2,https://github.com/torvalds/linux,CVE-2017-15951,CWE-20
24250,pgalloc.h,C,pgd_alloc,/*\n * S390 version\n * Copyright IBM Corp...,/*\n * S390 version\n * Copyright IBM Corp...,static inline pgd_t *pgd_alloc(struct mm_struc...,static inline pgd_t *pgd_alloc(struct mm_struc...,"{'added': [(103, '\tunsigned long *table = crs...",19,5,96.0,682.0,7,49,1,https://github.com/torvalds/linux,CVE-2016-2143,CWE-20


In [288]:
import ast
result_df.cwe_id = result_df.cwe_id.apply(ast.literal_eval)

def remove_duplicates(lst):
    return list(set(lst))

result_df.cwe_id = result_df.cwe_id.apply(remove_duplicates)

In [289]:
cwe_top_25 = [787, 79, 89, 416, 78, 20, 125, 22, 352, 434, 862, 476, 287, 190, 502, 77, 119, 798, 918, 306, 362, 269, 94, 863, 276]

result_df = result_df[result_df['cwe_id'].apply(lambda x: any(f'CWE-{i}' in x for i in cwe_top_25))]
result_df.shape

(145, 14)

In [290]:
result_df.head()

,file_name,method_name,code_before,code_after,func_before,func_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,repo,cve_id,cwe_id
1,signature-v4-utils_test.go,TestCheckValid,"// Copyright (c) 2015-2021 MinIO, Inc.\n//\n//...","// Copyright (c) 2015-2021 MinIO, Inc.\n//\n//...","func TestCheckValid(t *testing.T) {\n\tctx, ca...","func TestCheckValid(t *testing.T) {\n\tctx, ca...","{'added': [(79, '\tglobalIAMSys.CreateUser(ctx...",1.0,1.0,235.0,1475.0,https://github.com/minio/minio,CVE-2021-43858,[CWE-863]
2,extract.go,Extract,"/*\n * Copyright (c) SAS Institute, Inc.\n *\n...","/*\n * Copyright (c) SAS Institute, Inc.\n *\n...","func Extract(rs io.Reader, dest string) error ...","func Extract(rs io.Reader, dest string) error ...","{'added': [(24, '\t""path/filepath""'), (25, '\t...",17.0,8.0,118.0,681.0,https://github.com/sassoftware/go-rpmutils,CVE-2020-7667,[CWE-22]
6,criteria.go,OrderBy,// Package criteria implements a Criteria API ...,// Package criteria implements a Criteria API ...,func (c Criteria) OrderBy() string {\n\tif c.S...,func (c Criteria) OrderBy() string {\n\tif c.S...,"{'added': [(31, '\t\tlog.Error(""Invalid field ...",7.0,3.0,93.0,515.0,https://github.com/navidrome/navidrome,CVE-2022-23857,[CWE-89]
13,casa.go,GetServerAppInfo,"package service\n\nimport (\n\tjson2 ""encoding...","package service\n\nimport (\n\tjson2 ""encoding...","func (o *casaService) GetServerAppInfo(id, t s...","func (o *casaService) GetServerAppInfo(id, t s...","{'added': [(16, '\tGetServerList(index, size, ...",7.0,7.0,93.0,850.0,https://github.com/IceWhaleTech/CasaOS,CVE-2022-24193,[CWE-78]
14,casa.go,GetServerList,"package service\n\nimport (\n\tjson2 ""encoding...","package service\n\nimport (\n\tjson2 ""encoding...","func (o *casaService) GetServerList(index, siz...","func (o *casaService) GetServerList(index, siz...","{'added': [(16, '\tGetServerList(index, size, ...",7.0,7.0,93.0,850.0,https://github.com/IceWhaleTech/CasaOS,CVE-2022-24193,[CWE-78]


In [291]:
result_df = result_df.reset_index(drop=True)

## Only for Python

In [210]:
def remove_indent(code):
    lines = code.split('\n')
    min_indent = float('inf')
    for line in lines:
        stripped_line = line.lstrip()
        if stripped_line:  # Non-empty line
            indent = len(line) - len(stripped_line)
            min_indent = min(min_indent, indent)

    cleaned_lines = [line[min_indent:] for line in lines]
    cleaned_code = '\n'.join(cleaned_lines)
    return cleaned_code

In [211]:
result_df.func_before = result_df.func_before.apply(remove_indent)
result_df.func_after = result_df.func_after.apply(remove_indent)

In [212]:
def remove_print_statements(code):
    lines = code.split('\n')
    modified_lines = [line for line in lines if not line.strip().startswith('print')]
    modified_code = '\n'.join(modified_lines)
    return modified_code

def is_compilable(code):
    try:
        compile(code, "<string>", "exec")
        return True
    except SyntaxError as e:
        if "Missing parentheses in call to 'print'" in str(e):
            modified_code = remove_print_statements(code)
            try:
                compile(modified_code, "<string>", "exec")
                return True
            except SyntaxError:
                return False
        else:
            return False


In [213]:
not_compilable = []
for i in range(result_df.shape[0]):
    row = result_df.iloc[i]
    c1, c2, c3, c4 = row['code_before'], row['code_after'], row['func_before'], row['func_after']
    if not all(is_compilable(code) for code in [c1, c2, c3, c4]):
        not_compilable.append(i)

<string>:591: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:591: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:591: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:591: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:591: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:591: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:591: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:591: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:334: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:352: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:390: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:438: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:496: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:547: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:616: SyntaxWarning: "is" 

In [215]:
len(not_compilable)

51

In [217]:
result_df.drop(not_compilable, inplace=True)

In [218]:
result_df.shape

(587, 14)

In [220]:
result_df = result_df.reset_index(drop=True)

In [244]:
result_df.head()

,file_name,method_name,code_before,code_after,func_before,func_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,repo,cve_id,cwe_id
0,open_utils.c,read_new_config_info,//////////////////////////////////////////////...,//////////////////////////////////////////////...,static int read_new_config_info (WavpackContex...,static int read_new_config_info (WavpackContex...,"{'added': [(563, ' if (bytecnt >= 2) {'), (...",5.0,1.0,825.0,6305.0,https://github.com/dbry/WavPack,CVE-2016-10169,[CWE-125]
1,route.c,inet_rtm_getroute,/*\n * INET\t\tAn implementation of the TCP/IP...,/*\n * INET\t\tAn implementation of the TCP/IP...,static int inet_rtm_getroute(struct sk_buff *i...,static int inet_rtm_getroute(struct sk_buff *i...,"{'added': [(2766, '\tif (rtm->rtm_flags & RTM_...",9.0,2.0,2327.0,14667.0,https://github.com/torvalds/linux,CVE-2017-13686,[CWE-476]
2,main.c,mp4boxMain,/*\n *\t\t\tGPAC - Multimedia Framework C SDK\...,/*\n *\t\t\tGPAC - Multimedia Framework C SDK\...,"int mp4boxMain(int argc, char **argv)\n{\n\tu3...","int mp4boxMain(int argc, char **argv)\n{\n\tu3...","{'added': [(5598, '\t\t\tfprintf(stderr, ""File...",1.0,1.0,5787.0,43960.0,https://github.com/gpac/gpac,CVE-2020-23930,[CWE-476]
3,udp.c,udpv6_sendmsg,/*\n *\tUDP over IPv6\n *\tLinux INET6 impleme...,/*\n *\tUDP over IPv6\n *\tLinux INET6 impleme...,"int udpv6_sendmsg(struct sock *sk, struct msgh...","int udpv6_sendmsg(struct sock *sk, struct msgh...","{'added': [(1113, '\tstruct ipv6_txoptions *op...",6.0,2.0,1221.0,7876.0,https://github.com/torvalds/linux,CVE-2016-3841,"[CWE-264, CWE-416]"
4,api.c,crypto_larval_lookup,/*\n * Scatterlist Cryptographic API.\n *\n * ...,/*\n * Scatterlist Cryptographic API.\n *\n * ...,struct crypto_alg *crypto_larval_lookup(const ...,struct crypto_alg *crypto_larval_lookup(const ...,"{'added': [(219, '\t\trequest_module(""crypto-%...",2.0,2.0,424.0,2423.0,https://github.com/torvalds/linux,CVE-2013-7421,[CWE-269]


## For the rest

In [239]:
result_df.cwe_id.value_counts()

cwe_id
[CWE-125]             547
[CWE-119]             470
[CWE-20]              273
[CWE-476]             257
[CWE-787]             242
                     ... 
[CWE-787, CWE-681]      1
[CWE-476, CWE-362]      1
[CWE-125, CWE-362]      1
[CWE-125, CWE-200]      1
[CWE-125, CWE-190]      1
Name: count, Length: 73, dtype: int64

In [292]:
result_df.to_csv('CVEFixes_go_25.csv', index=False)

In [293]:
from collections import Counter

counters = Counter()

for sublist in result_df['cwe_id']:
    counters.update(sublist)

# Display the counts for each element
counters

Counter({'CWE-863': 36,
         'CWE-190': 19,
         'CWE-78': 13,
         'CWE-918': 12,
         'CWE-22': 11,
         'CWE-20': 10,
         'CWE-89': 9,
         'CWE-79': 9,
         'CWE-287': 9,
         'CWE-276': 4,
         'CWE-269': 4,
         'CWE-362': 3,
         'CWE-798': 2,
         'CWE-787': 1,
         'CWE-125': 1,
         'CWE-862': 1,
         'CWE-476': 1})

In [227]:
result_df.num_tokens_in_file.describe()

count      587.000000
mean      8708.102215
std      10512.587620
min         48.000000
25%       1116.000000
50%       3938.000000
75%      13180.000000
max      39494.000000
Name: num_tokens_in_file, dtype: float64

In [304]:
import datasets
from datasets import Dataset, DatasetDict

hfC = Dataset.from_pandas(pd.read_csv('assets/CVEFixes_c_25.csv'))
hfG = Dataset.from_pandas(pd.read_csv('assets/CVEFixes_go_25.csv'))
hfJ = Dataset.from_pandas(pd.read_csv('assets/CVEFixes_java_25.csv'))
hfP = Dataset.from_pandas(pd.read_csv('assets/CVEFixes_python_25.csv'))
hfR = Dataset.from_pandas(pd.read_csv('assets/CVEFixes_ruby_25.csv'))

In [308]:
from datasets import DatasetDict

ddict = DatasetDict({
    "c": hfC,
    "go": hfG,
    "java": hfJ,
    "python": hfP,
    "ruby": hfR,
})

ddict.push_to_hub("euisuh15/cveFixes")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]


KeyboardInterrupt



In [310]:
ddict.push_to_hub('euisuh15/cveFixes1')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/euisuh15/cveFixes1/commit/f5b2212b9c568cb087362e62a0ea05a09cce842d', commit_message='Upload dataset', commit_description='', oid='f5b2212b9c568cb087362e62a0ea05a09cce842d', pr_url=None, pr_revision=None, pr_num=None)

# CVEFixes by language

## Retrieve dataset from HF

In [2]:
import pandas as pd
from datasets import load_dataset
import os

os.environ["HF_HOME"] = "/tmp/huggingface"
# dataset = load_dataset("euisuh15/cveFixes1")
dataset = load_dataset("euisuh15/cveFixes1", split='c')

# lang option:  c | go | java | python | ruby
# lang = "c"
# df = pd.DataFrame(dataset[lang])
df = pd.DataFrame(dataset)

df.shape

/home/local/QCRI/ejeong/anaconda2/envs/vllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(2621, 14)

In [3]:
df.head()

,file_name,method_name,code_before,code_after,func_before,func_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,repo,cve_id,cwe_id
0,open_utils.c,read_new_config_info,//////////////////////////////////////////////...,//////////////////////////////////////////////...,static int read_new_config_info (WavpackContex...,static int read_new_config_info (WavpackContex...,"{'added': [(563, ' if (bytecnt >= 2) {'), (...",5.0,1.0,825.0,6305.0,https://github.com/dbry/WavPack,CVE-2016-10169,['CWE-125']
1,route.c,inet_rtm_getroute,/*\n * INET\t\tAn implementation of the TCP/IP...,/*\n * INET\t\tAn implementation of the TCP/IP...,static int inet_rtm_getroute(struct sk_buff *i...,static int inet_rtm_getroute(struct sk_buff *i...,"{'added': [(2766, '\tif (rtm->rtm_flags & RTM_...",9.0,2.0,2327.0,14667.0,https://github.com/torvalds/linux,CVE-2017-13686,['CWE-476']
2,main.c,mp4boxMain,/*\n *\t\t\tGPAC - Multimedia Framework C SDK\...,/*\n *\t\t\tGPAC - Multimedia Framework C SDK\...,"int mp4boxMain(int argc, char **argv)\n{\n\tu3...","int mp4boxMain(int argc, char **argv)\n{\n\tu3...","{'added': [(5598, '\t\t\tfprintf(stderr, ""File...",1.0,1.0,5787.0,43960.0,https://github.com/gpac/gpac,CVE-2020-23930,['CWE-476']
3,udp.c,udpv6_sendmsg,/*\n *\tUDP over IPv6\n *\tLinux INET6 impleme...,/*\n *\tUDP over IPv6\n *\tLinux INET6 impleme...,"int udpv6_sendmsg(struct sock *sk, struct msgh...","int udpv6_sendmsg(struct sock *sk, struct msgh...","{'added': [(1113, '\tstruct ipv6_txoptions *op...",6.0,2.0,1221.0,7876.0,https://github.com/torvalds/linux,CVE-2016-3841,"['CWE-264', 'CWE-416']"
4,api.c,crypto_larval_lookup,/*\n * Scatterlist Cryptographic API.\n *\n * ...,/*\n * Scatterlist Cryptographic API.\n *\n * ...,struct crypto_alg *crypto_larval_lookup(const ...,struct crypto_alg *crypto_larval_lookup(const ...,"{'added': [(219, '\t\trequest_module(""crypto-%...",2.0,2.0,424.0,2423.0,https://github.com/torvalds/linux,CVE-2013-7421,['CWE-269']


In [4]:
df.to_csv('vul7.csv')